#### 一. 启停服务
1. 开启管理界面
    1. 启动插件:  
    ```bash
     $ rabbitmq-plugins enable rabbitmq_management
    ```
    
    2. (可选)增加用户admin
    ```bash
    # 用户名:admin, 密码: admin 
    $ rabbitmqctl add_user admin admin
    
    # 设置admin用户的管理员权限
    $ rabbitmqctl set_user_tags admin administrator
    
     # 显示用户列表和权限
    $ rabbitmqctl list_users
    Listing users ...
    user    tags
    admin   [administrator]
    guest   [administrator]
    ```
    2. 端口占用`15672`: WEB管理页面  
     broker会默认创建一个`guest`用户, 密码也是`guest`
2. 开启`rabbitmq-server`   
   ```bash
   #service服务
   service rabbitmq-server start
   #解压缩
   sbin/rabbitmq-server
   #停止服务
   sbin/rabbitmqctl shutdown
   ```
     
#### 二. 修改最大文件打开个数   
   rabbitmq主要修改配置是文件最大打开个数, 以此应对更多的客户端连接; 有2个维度的配置:
1. `ulimit -n`: 每个用户允许的最大文件打开个数 
    * 查看
    ```bash
    $ ulimit -n
    1700000
    ```

    * 设置
        * **临时性**：  
          通过`ulimit -Sn`设置最大打开文件描述符数的`soft limit`. 注意`soft limit`不能大于`hard limit`（ulimit -Hn可查看hard limit），另外`ulimit -n`默认查看的是`soft limit`，但是`ulimit -n 1800000`则是同时设置`soft limit`和`hard limit`。对于非root用户只能设置比原来小的hard limit。
            * 查看hard limit：
            ```bash
            $ ulimit -Hn
            1700000
            ```

            * 设置soft limit，必须小于hard limit：
            ```bash
            $ ulimit -Sn 1600000
            ```

        * **永久性**：   
         上面的方法只是临时性的，注销重新登录就失效了，而且不能增大hard limit，只能在hard limit范围内修改soft limit。若要使修改永久有效，则需要在`/etc/security/limits.conf`中进行设置（需要root权限），可添加如下两行，表示用户chanon最大打开文件描述符数的`soft limit`为1800000，`hard limit`为2000000。以下设置需要注销之后重新登录才能生效：
        ```text
        chanon           soft    nofile          1800000
        chanon           hard   nofile          2000000
        ```

         设置nofile的`hard limit`还有一点要注意的就是hard limit不能大于`/proc/sys/fs/nr_open`，假如hard limit大于nr_open，注销后无法正常登录。可以修改`nr_open`的值：
        ```bash
        $ echo 2000000 > /proc/sys/fs/nr_open
        # 推荐至少配置成65536对于用户`rabbitmq`的fd个数, 一般情况下4096就银镜足够
        ```

2. `fs.file-max` : os kernel允许的最大文件打开个数  
    * 查看
    ```bash
    $ cat /proc/sys/fs/file-max
    186405
    ```
    * 设置
        * **临时性**
        ```bash
        # echo 1000000 > /proc/sys/fs/file-max
        ```
        * **永久性**：在/etc/sysctl.conf中设置
        ```bash
        fs.file-max = 1000000
        ```


#### 三. 日志文件和管理
1. rabbitmq日志文件默认位置在: `/var/log/rabbitmq`
2. 日志只会以追加形式记录, 默认每周对日志文件rotate.   
   采用工具[logrotate](https://linux.die.net/man/8/logrotate)进行日志rotate, rotate的策略在`/etc/logrotate.d/rabbitmq-server`中配置

#### 四. [Production Checklist](https://www.rabbitmq.com/production-checklist.html)
1. **Virtual Hosts**   
    * 单租户(`single-tenant`)场景下, 使用默认的`/` virtual host即可
    * 多租户(`multi-tenant`)下, 应该使用不同的`vhost`; 比如`project1_development`, `project1_production`, `project2_development`, `project2_production`等等

2. **Users**   
   生产环境下, 应该为每个`application`设置一个用户. 比如你分别有`a mobile app`, `a Web app`, and `a data aggregation system`, 就应该为这3个应用设置3个用户
   
3. **内存**   
    * 默认情况下, 当rabbitmq占用内存超过全部内存的40%`{vm_memory_high_watermark, 0.4}`时,将不会再继续接受任何消息. 这是为了预留足够的空间进行`os swapping`. 如果占用内存过大反而可能会导致rabbitmq的性能下降甚至死亡
    * `vm_memory_high_watermark`可以配置成`0.40到0.66`. 超过0.7是不允许的. 至少要预留30%的内存, 否则会因为`os paging`造成rabbitmq性能严重下降

4. **磁盘**  